<a href="https://colab.research.google.com/github/safal207/hello-world/blob/master/%D0%9E%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%B3%D0%B8%D0%BF%D0%B5%D1%80%D0%BF%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D0%BE%D0%B2_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%B5%D1%82%D0%B8_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_Keras_Tuner_%22fashion_mnist_keras_tuner_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оптимизация гиперпараметров нейросети с помощью [Keras Tuner](https://github.com/keras-team/keras-tuner)



Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

Не забудьте подключить GPU, чтобы сеть обучалась быстрее (Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU).



## Гиперпараметры обучения нейронной сети

- Количество слоев нейронной сети
- Количество нейронов в каждом слое
- Функции активации, которые используются в слоях
- Тип оптимизатора при обучении нейронной сети
- Количество эпох обучения

## Установка Keras Tuner

In [ ]:
pip install -U keras-tuner

     |████████████████████████████████| 61kB 3.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=4aa4417dbceeff4f2e2f2c874ae8f5361e6a14cd7ce55c297ba2a6852330247d
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=f579b2ad146aa2b9c2074bc21e40a41f36f4040c3efc1aebd25903934b16765c
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


## Подключаем нужные пакеты

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from google.colab import files
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np

## Подготовка данных для обучения сети

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train / 255 
x_test = x_test / 255 
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

## Задаем функцию создания нейронной сети

In [ ]:
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])    
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                   min_value=512,    # минимальное количество нейронов - 128
                                   max_value=1024,   # максимальное количество - 1024
                                   step=32),
                    input_dim=784,
                    activation=activation_choice))
    model.add(Dense(units=hp.Int('units_hidden',        
                                   min_value=128,   
                                   max_value=600,   
                                   step=32),
                    activation=activation_choice))   
    
    model.add(Dense(units=hp.Int('units_hidden',        
                                   min_value=48,   
                                   max_value=300,   
                                   step=32),
                    activation=activation_choice))   
    
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

## Создаем tuner

Доступные типы тюнеров: 
- RandomSearch - случайный поиск.
- Hyperband - алгоритм оптимизации на основе многорукого бандита, Li, Lisha, and Kevin Jamieson. ["Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization."Journal of Machine Learning Research 18 (2018): 1-52](http://jmlr.org/papers/v18/16-558.html).
- BayesianOptimization - [байесовская оптимизация](https://en.wikipedia.org/wiki/Bayesian_optimization).

In [ ]:
tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=120,               # максимальное количество запусков обучения 
    directory='test_directory'   # каталог, куда сохраняются обученные сети  
    )

## Запускаем подбор гиперпараметров

Пространство поиска

In [ ]:
tuner.search_space_summary()

Подбор гиперпараметров

In [ ]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=20,                # Количество эпох обучения 
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Epoch 1/20
188/188 [==============================] - 2s 11ms/step - loss: 0.9324 - accuracy: 0.6966 - val_loss: 0.5372 - val_accuracy: 0.7947
Epoch 2/20
188/188 [==============================] - 2s 10ms/step - loss: 0.5233 - accuracy: 0.8044 - val_loss: 0.4433 - val_accuracy: 0.8311
Epoch 3/20
188/188 [==============================] - 2s 10ms/step - loss: 0.4383 - accuracy: 0.8357 - val_loss: 0.4592 - val_accuracy: 0.8382
Epoch 4/20
188/188 [==============================] - 2s 10ms/step - loss: 0.3896 - accuracy: 0.8543 - val_loss: 0.4779 - val_accuracy: 0.8131
Epoch 5/20
188/188 [==============================] - 2s 9ms/step - loss: 0.3542 - accuracy: 0.8665 - val_loss: 0.4208 - val_accuracy: 0.8448
Epoch 6/20
188/188 [==============================] - 2s 9ms/step - loss: 0.3300 - accuracy: 0.8741 - val_loss: 0.3723 - val_accuracy: 0.8685
Epoch 7/20
188/188 [==============================] - 2s 10ms/step - loss: 0.3095 - accuracy: 0.8818 - val_loss: 0.3600 - val_accuracy: 0.8751
E

Epoch 1/20
188/188 [==============================] - 2s 9ms/step - loss: 0.6818 - accuracy: 0.7447 - val_loss: 0.6983 - val_accuracy: 0.7731
Epoch 2/20
188/188 [==============================] - 2s 9ms/step - loss: 0.4352 - accuracy: 0.8373 - val_loss: 0.4228 - val_accuracy: 0.8457
Epoch 3/20
188/188 [==============================] - 2s 9ms/step - loss: 0.3769 - accuracy: 0.8605 - val_loss: 0.3874 - val_accuracy: 0.8637
Epoch 4/20
188/188 [==============================] - 2s 8ms/step - loss: 0.3414 - accuracy: 0.8722 - val_loss: 0.3858 - val_accuracy: 0.8552
Epoch 5/20
188/188 [==============================] - 2s 9ms/step - loss: 0.3168 - accuracy: 0.8814 - val_loss: 0.3339 - val_accuracy: 0.8849
Epoch 6/20
188/188 [==============================] - 2s 9ms/step - loss: 0.2948 - accuracy: 0.8894 - val_loss: 0.3515 - val_accuracy: 0.8795
Epoch 7/20
188/188 [==============================] - 2s 9ms/step - loss: 0.2840 - accuracy: 0.8934 - val_loss: 0.3454 - val_accuracy: 0.8802
Epoch 

Epoch 1/20
188/188 [==============================] - 2s 9ms/step - loss: 0.6872 - accuracy: 0.7478 - val_loss: 0.5381 - val_accuracy: 0.8091
Epoch 2/20
188/188 [==============================] - 2s 9ms/step - loss: 0.4372 - accuracy: 0.8375 - val_loss: 0.4321 - val_accuracy: 0.8357
Epoch 3/20
188/188 [==============================] - 2s 8ms/step - loss: 0.3763 - accuracy: 0.8589 - val_loss: 0.5453 - val_accuracy: 0.8190
Epoch 4/20
188/188 [==============================] - 2s 9ms/step - loss: 0.3470 - accuracy: 0.8713 - val_loss: 0.4257 - val_accuracy: 0.8438
Epoch 5/20
188/188 [==============================] - 2s 9ms/step - loss: 0.3190 - accuracy: 0.8810 - val_loss: 0.3896 - val_accuracy: 0.8681
Epoch 6/20
188/188 [==============================] - 2s 9ms/step - loss: 0.2978 - accuracy: 0.8872 - val_loss: 0.3448 - val_accuracy: 0.8808
Epoch 7/20
188/188 [==============================] - 2s 8ms/step - loss: 0.2804 - accuracy: 0.8948 - val_loss: 0.3559 - val_accuracy: 0.8808
Epoch 

Epoch 1/20
188/188 [==============================] - 1s 8ms/step - loss: 1.0648 - accuracy: 0.6693 - val_loss: 0.7356 - val_accuracy: 0.7569
Epoch 2/20
188/188 [==============================] - 1s 7ms/step - loss: 0.6688 - accuracy: 0.7794 - val_loss: 0.6127 - val_accuracy: 0.7919
Epoch 3/20
188/188 [==============================] - 1s 7ms/step - loss: 0.5821 - accuracy: 0.8038 - val_loss: 0.5591 - val_accuracy: 0.8062
Epoch 4/20
188/188 [==============================] - 1s 7ms/step - loss: 0.5371 - accuracy: 0.8173 - val_loss: 0.5304 - val_accuracy: 0.8109
Epoch 5/20
188/188 [==============================] - 1s 7ms/step - loss: 0.5086 - accuracy: 0.8250 - val_loss: 0.5026 - val_accuracy: 0.8233
Epoch 6/20
188/188 [==============================] - 1s 7ms/step - loss: 0.4887 - accuracy: 0.8314 - val_loss: 0.4859 - val_accuracy: 0.8272
Epoch 7/20
188/188 [==============================] - 1s 7ms/step - loss: 0.4736 - accuracy: 0.8355 - val_loss: 0.4748 - val_accuracy: 0.8292
Epoch 

Epoch 1/20
188/188 [==============================] - 2s 10ms/step - loss: 0.5464 - accuracy: 0.8018 - val_loss: 0.3998 - val_accuracy: 0.8555
Epoch 2/20
188/188 [==============================] - 2s 9ms/step - loss: 0.3778 - accuracy: 0.8616 - val_loss: 0.3892 - val_accuracy: 0.8602
Epoch 3/20
188/188 [==============================] - 2s 9ms/step - loss: 0.3442 - accuracy: 0.8730 - val_loss: 0.3714 - val_accuracy: 0.8602
Epoch 4/20
188/188 [==============================] - 2s 10ms/step - loss: 0.3159 - accuracy: 0.8819 - val_loss: 0.3480 - val_accuracy: 0.8745
Epoch 5/20
188/188 [==============================] - 2s 10ms/step - loss: 0.2912 - accuracy: 0.8924 - val_loss: 0.3399 - val_accuracy: 0.8753
Epoch 6/20
188/188 [==============================] - 2s 9ms/step - loss: 0.2748 - accuracy: 0.8961 - val_loss: 0.3226 - val_accuracy: 0.8791
Epoch 7/20
188/188 [==============================] - 2s 9ms/step - loss: 0.2590 - accuracy: 0.9020 - val_loss: 0.3206 - val_accuracy: 0.8878
Epo

...

Epoch 1/20
188/188 [==============================] - 1s 8ms/step - loss: 1.4274 - accuracy: 0.6087 - val_loss: 0.9152 - val_accuracy: 0.7145
Epoch 2/20
188/188 [==============================] - 1s 7ms/step - loss: 0.7976 - accuracy: 0.7452 - val_loss: 0.6996 - val_accuracy: 0.7803
Epoch 3/20
188/188 [==============================] - 1s 7ms/step - loss: 0.6588 - accuracy: 0.7870 - val_loss: 0.6252 - val_accuracy: 0.7938
Epoch 4/20
188/188 [==============================] - 1s 7ms/step - loss: 0.5892 - accuracy: 0.8061 - val_loss: 0.5663 - val_accuracy: 0.8084
Epoch 5/20
188/188 [==============================] - 1s 7ms/step - loss: 0.5477 - accuracy: 0.8171 - val_loss: 0.5431 - val_accuracy: 0.8117
Epoch 6/20
188/188 [==============================] - 1s 7ms/step - loss: 0.5191 - accuracy: 0.8256 - val_loss: 0.5134 - val_accuracy: 0.8230
Epoch 7/20
188/188 [==============================] - 1s 7ms/step - loss: 0.4993 - accuracy: 0.8300 - val_loss: 0.4943 - val_accuracy: 0.8263
Epoch 

Epoch 1/20
188/188 [==============================] - 2s 9ms/step - loss: 0.6426 - accuracy: 0.7940 - val_loss: 0.4120 - val_accuracy: 0.8487
Epoch 2/20
188/188 [==============================] - 1s 8ms/step - loss: 0.4054 - accuracy: 0.8535 - val_loss: 0.4055 - val_accuracy: 0.8531
Epoch 3/20
188/188 [==============================] - 1s 8ms/step - loss: 0.3593 - accuracy: 0.8671 - val_loss: 0.4125 - val_accuracy: 0.8482
Epoch 4/20
188/188 [==============================] - 1s 8ms/step - loss: 0.3431 - accuracy: 0.8734 - val_loss: 0.3586 - val_accuracy: 0.8725
Epoch 5/20
188/188 [==============================] - 2s 8ms/step - loss: 0.3121 - accuracy: 0.8849 - val_loss: 0.3330 - val_accuracy: 0.8763
Epoch 6/20
188/188 [==============================] - 1s 8ms/step - loss: 0.2914 - accuracy: 0.8921 - val_loss: 0.3884 - val_accuracy: 0.8627
Epoch 7/20
188/188 [==============================] - 1s 7ms/step - loss: 0.2788 - accuracy: 0.8948 - val_loss: 0.3700 - val_accuracy: 0.8708
Epoch 

Epoch 1/20
188/188 [==============================] - 2s 9ms/step - loss: 0.7310 - accuracy: 0.7509 - val_loss: 0.5547 - val_accuracy: 0.8018
Epoch 2/20
188/188 [==============================] - 1s 8ms/step - loss: 0.5154 - accuracy: 0.8213 - val_loss: 0.4882 - val_accuracy: 0.8283
Epoch 3/20
188/188 [==============================] - 1s 8ms/step - loss: 0.4732 - accuracy: 0.8331 - val_loss: 0.4605 - val_accuracy: 0.8382
Epoch 4/20
188/188 [==============================] - 1s 8ms/step - loss: 0.4515 - accuracy: 0.8422 - val_loss: 0.4504 - val_accuracy: 0.8406
Epoch 5/20
188/188 [==============================] - 1s 8ms/step - loss: 0.4372 - accuracy: 0.8452 - val_loss: 0.4475 - val_accuracy: 0.8389
Epoch 6/20
188/188 [==============================] - 1s 8ms/step - loss: 0.4253 - accuracy: 0.8505 - val_loss: 0.4477 - val_accuracy: 0.8388
Epoch 7/20
188/188 [==============================] - 1s 8ms/step - loss: 0.4162 - accuracy: 0.8545 - val_loss: 0.4167 - val_accuracy: 0.8539
Epoch 

Epoch 1/20
188/188 [==============================] - 2s 9ms/step - loss: 0.5332 - accuracy: 0.8099 - val_loss: 0.4098 - val_accuracy: 0.8544
Epoch 2/20
188/188 [==============================] - 2s 9ms/step - loss: 0.3671 - accuracy: 0.8654 - val_loss: 0.3875 - val_accuracy: 0.8551
Epoch 3/20
188/188 [==============================] - 2s 9ms/step - loss: 0.3256 - accuracy: 0.8787 - val_loss: 0.3428 - val_accuracy: 0.8786
Epoch 4/20
188/188 [==============================] - 2s 8ms/step - loss: 0.2941 - accuracy: 0.8904 - val_loss: 0.3425 - val_accuracy: 0.8788
Epoch 5/20
188/188 [==============================] - 2s 9ms/step - loss: 0.2817 - accuracy: 0.8951 - val_loss: 0.3197 - val_accuracy: 0.8853
Epoch 6/20
188/188 [==============================] - 2s 8ms/step - loss: 0.2619 - accuracy: 0.9014 - val_loss: 0.3743 - val_accuracy: 0.8680
Epoch 7/20
188/188 [==============================] - 2s 8ms/step - loss: 0.2535 - accuracy: 0.9052 - val_loss: 0.3023 - val_accuracy: 0.8911
Epoch 

INFO:tensorflow:Oracle triggered exit


## Выбираем лучшую модель

In [ ]:
tuner.results_summary()

Получаем три лучших модели

In [ ]:
models = tuner.get_best_models(num_models=3)

Оцениваем качество модели на тестовых данных

In [ ]:
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 928)               728480    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               475648    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,471,914
Trainable params: 1,471,914
Non-trainable params: 0
_________________________________________________________________
313/313 [==============================] - 1s 4ms/step - loss: 0.3788 - accuracy: 0.8931

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
de